### Planetscope preprocessing - Reflectance Mosaic



This script:
- Reads multiple Planetscope images/scenes
- Convert TOA Radiance to TOA Reflectance
- Creates a composite image (or mosaic) 
- Clips mosaic over selected area of interest (AOI)

Main packages used are rasterio and fiona.

Code is based on notebooks from Planetlabs, available here: https://github.com/planetlabs/notebooks

Data used are from Planetscope 21. June 2019 over a selected study area in the Brazilian Amazon.

Available export options:

- Reflectance images scaled - *(default setting)*
- Reflectance mosaic scaled - *(default setting)*
- Reflectance mosaic scaled clipped to AOI - *(default setting)*   





- Reflectance images unscaled
- Reflectance mosaic unscaled
- Reflectance mosaic unscaled clipped to AOI

***

In [1]:
import numpy as np
import matplotlib
import rasterio
import fiona

#### Step 1. Set up project folders - rest is automatic
The following input and directory structure is required:

- *Number of images/scenes*


- *Project folder*  -  all folders inside Project folder


- *AnalyticMS folder*  -  only the AnalyticMS.tif files, no unusable datamasks(udm)


- *Metadata folder*  -  only the metadata.xml files


- *AOI folder*  -  just one AOI file in GeoJSON format


- *Results folder*  -  receives export of reflectance images, also serves as input to the clip-functions that further clips the results to selected AOI


The default setting export scaled reflectance images, mosaic and clipped mosaic.

The unscaled images, mosaic and unscaled clipped mosaic are available by calling the *unscaled-functions* in Step 6, 7 and 9.

In [2]:
n_images = 7
Project_folder = 'planet_21/'
AnalyticMS_folder = 'planet_21/AnalyticMS_files/'
Metadata_folder = 'planet_21/Metadata_files/'
aoi_folder = 'planet_21/AOI/'
Results_folder = 'planet_21/Results/'

#### Step 2. Read and Inspect Images

In [3]:
# Use 7 scenes from 21. June 2019 for coversion to TOA Reflectance mosaic
# Collect image filenames 
from os import walk
AnalyticMS_files = []
for (dirpath, dirnames, filenames) in walk(AnalyticMS_folder):
    AnalyticMS_files.extend(filenames)
    break

# Add image file path
AnalyticMS_path = []
for i in AnalyticMS_files:
    AnalyticMS_path.append(AnalyticMS_folder + i)
    print(AnalyticMS_folder + i) # Verify correct images

planet_21/AnalyticMS_files/20190721_133241_1034_3B_AnalyticMS.tif
planet_21/AnalyticMS_files/20190721_133242_1034_3B_AnalyticMS.tif
planet_21/AnalyticMS_files/20190721_133243_1034_3B_AnalyticMS.tif
planet_21/AnalyticMS_files/20190721_133316_1044_3B_AnalyticMS.tif
planet_21/AnalyticMS_files/20190721_133317_1044_3B_AnalyticMS.tif
planet_21/AnalyticMS_files/20190721_133318_1044_3B_AnalyticMS.tif
planet_21/AnalyticMS_files/20190721_133319_1044_3B_AnalyticMS.tif


In [4]:
# Reading image metadata
Analytic_meta_files = []
for (dirpath, dirnames, filenames) in walk(Metadata_folder):
    Analytic_meta_files.extend(filenames)
    break

# Add image metadata file path
Analytic_meta_path = []
for i in Analytic_meta_files:
    Analytic_meta_path.append(Metadata_folder + i)
    print(Metadata_folder + i) # Verify correct metadata

planet_21/Metadata_files/20190721_133241_1034_3B_AnalyticMS_metadata.xml
planet_21/Metadata_files/20190721_133242_1034_3B_AnalyticMS_metadata.xml
planet_21/Metadata_files/20190721_133243_1034_3B_AnalyticMS_metadata.xml
planet_21/Metadata_files/20190721_133316_1044_3B_AnalyticMS_metadata.xml
planet_21/Metadata_files/20190721_133317_1044_3B_AnalyticMS_metadata.xml
planet_21/Metadata_files/20190721_133318_1044_3B_AnalyticMS_metadata.xml
planet_21/Metadata_files/20190721_133319_1044_3B_AnalyticMS_metadata.xml


In [5]:
# Loading images into list using rasterio
img_list = []
for image in AnalyticMS_path:
    img_list.append(rasterio.open(image))
print('Length of img_list: {} '.format(len(img_list)))

Length of img_list: 7 


At this point we can use rasterio to inspect the metadata of these three images. Specifically, in order to create a composite from these images, we want to verify that all three images have the same data type, the same coordinate reference systems and the same band count

In [6]:
# Inspecting image metadata
print("Image: dtype | crs | band count")
for image in img_list:
    print(image.meta['dtype'], image.meta['crs'], image.meta['count'])

Image: dtype | crs | band count
uint16 EPSG:32722 4
uint16 EPSG:32722 4
uint16 EPSG:32722 4
uint16 EPSG:32722 4
uint16 EPSG:32722 4
uint16 EPSG:32722 4
uint16 EPSG:32722 4


Let's take a closer look at what these bands contain:

In [7]:
# Read in color interpretations of each band in img1 - assume same values for rest of the images
colors = [img_list[0].colorinterp[band] for band in range(img_list[0].count)]

# taking a look at img1's band types:
for color in colors:
    print(color.name)

blue
green
red
undefined


The fourth channel is a binary alpha mask: this is common in satellite color models, and can be confirmed in Planet's documentation on the PSSCene3Band product.

Now that we've verified all three satellite images have the same critical metadata, we can safely use rasterio.merge to stitch them together.

#### Step 3. Extract the Data from Each Spectral Band
In this step, Rasterio (a Python library for reading and writing geospatial raster datasets) is used to open the raster images (the .tif files). 

Then, the band data will is extracted and loaded into arrays for further manipulation with Python's NumPy libary.

Note: in PlanetScope 4-band images, the band order is BGRN: (1) Blue, (2) Green, (3) Red, (4) Near-infrared.

In [8]:
# Radiance values are loaded into lists
band_blue_radiance = [0]*n_images
band_green_radiance = [0]*n_images
band_red_radiance = [0]*n_images
band_nir_radiance = [0]*n_images

# Using rasterio to read radiance values for the images
for i in range(n_images):
    with rasterio.open(AnalyticMS_path[i]) as src:
        band_blue_radiance[i] = src.read(1)
    with rasterio.open(AnalyticMS_path[i]) as src:
        band_green_radiance[i] = src.read(2)
    with rasterio.open(AnalyticMS_path[i]) as src:
        band_red_radiance[i] = src.read(3)
    with rasterio.open(AnalyticMS_path[i]) as src:
        band_nir_radiance[i] = src.read(4)

#### Step 4. Extract the Coefficients
Before converting to reflectance, the conversion coefficients from the metadata file (the .xml file) must be extracted.

In [9]:
from xml.dom import minidom

# Gathering the coefficients for the 7 images
coeffs_list = [] 
for image in Analytic_meta_path:
    xmldoc = minidom.parse(image)
    nodes = xmldoc.getElementsByTagName("ps:bandSpecificMetadata")
    
    # XML parser refers to bands by numbers 1-4
    coeffs = {} # Coefficients for each image/scene
    for node in nodes:
        band_nr = node.getElementsByTagName("ps:bandNumber")[0].firstChild.data
        if band_nr in ['1', '2', '3', '4']:
            i = int(band_nr)
            value = node.getElementsByTagName("ps:reflectanceCoefficient")[0].firstChild.data
            coeffs[i] = float(value)
    coeffs_list.append(coeffs)

for coffee in coeffs_list:
    print (coffee) # Verify data/coffee

{1: 2.22787121429e-05, 2: 2.3597734536e-05, 3: 2.62801712705e-05, 4: 3.9553550288e-05}
{1: 2.22957954184e-05, 2: 2.36158292355e-05, 3: 2.63003228576e-05, 4: 3.95838798777e-05}
{1: 2.23129263982e-05, 2: 2.36339744636e-05, 3: 2.6320530717e-05, 4: 3.96142941613e-05}
{1: 2.22858931046e-05, 2: 2.36138635285e-05, 3: 2.63757407995e-05, 4: 3.9632589334e-05}
{1: 2.23029452396e-05, 2: 2.3631931765e-05, 3: 2.63959222968e-05, 4: 3.96629143589e-05}
{1: 2.23200437529e-05, 2: 2.36500491436e-05, 3: 2.64161586838e-05, 4: 3.96933218618e-05}
{1: 2.23372065248e-05, 2: 2.36682346096e-05, 3: 2.64364711218e-05, 4: 3.97238436401e-05}


Note that the coefficients are all of order 1e-5, and that the coefficient for NIR is significantly higher than the coefficient for blue. This is a big deal if your use case involves performing band math because a pixel with a NIR/blue ratio of 1.0 in the radiance image will have a NIR/blue ratio of 3.35/1.929=1.73 in the reflectance image.

Most spectral indices are defined in terms of reflectance, not radiance.

#### Step 5: Convert Radiance to Reflectance
Radiance is measured in SI units: $W/m^2$. Reflectance is a ratio from 0 to 1. The conversion is performed as a per-band scalar multiplication:

In [10]:
# Radiance values are loaded into lists
band_blue_reflectance = [];   blue_ref_scaled = [0]*n_images
band_green_reflectance = [];  green_ref_scaled = [0]*n_images
band_red_reflectance = [];    red_ref_scaled = [0]*n_images
band_nir_reflectance = [];    nir_ref_scaled = [0]*n_images

# Calculating reflectance from radiance and conversion coefficients
for i in range(n_images):
        band_blue_reflectance.append(band_blue_radiance[i] * coeffs_list[i][1])
        band_green_reflectance.append(band_green_radiance[i] * coeffs_list[i][2])
        band_red_reflectance.append(band_red_radiance[i] * coeffs_list[i][3])
        band_nir_reflectance.append(band_nir_radiance[i] * coeffs_list[i][4])
        
# Check the results by looking at the min-max range of the radiance (before) vs reflectance (after)
# for the red band from the first image, excluding NaN values
red_rad_min = np.nanmin(band_red_radiance[0]);    red_rad_max = np.nanmax(band_red_radiance[0])
red_ref_min = np.nanmin(band_red_reflectance[0]); red_ref_max = np.nanmax(band_red_reflectance[0])
print("Red band radiance goes from: {} to {}".format(red_rad_min, red_rad_max))
print("Red band reflectance goes from: {} to {}".format(red_ref_min, red_ref_max))

Red band radiance goes from: 0 to 10839
Red band reflectance goes from: 0.0 to 0.28485077640094947


#### Step 6. Write the Reflectance Images
A note: Reflectance is generally defined as a floating point number between 0 and 1, but image file formats are much more commonly stored as unsigned integers. A common practice in the industry is to multiply the reflectance value by a *scale factor* of 10,000, then save the result as a file with data type uint16.


Export of normal scaled GeoTIFF (multiplied with scale factor of 10 000 and stored as *dtype=uint16*)

In [11]:
# Function to export scaled images - (alternative option)
def scaled_export(images):
    for i in range(len(images)):
        # Get the metadata of original GeoTIFF:
        meta = images[i].meta
        
        # Set the source metadata as kwargs we'll use to write the new data:
        # update the 'dtype' value to uint16:
        kwargs = meta
        kwargs.update(dtype='uint16')
        
        # As noted above, scale reflectance value by a factor of 10k:
        scale = 10000
        blue_ref_scaled[i] = scale * band_blue_reflectance[i]
        green_ref_scaled[i] = scale * band_green_reflectance[i]
        red_ref_scaled[i] = scale * band_red_reflectance[i]
        nir_ref_scaled[i] = scale * band_nir_reflectance[i]
        
        # Compute new min & max values for the scaled red band, just for comparison
        red_min_scaled = np.amin(red_ref_scaled[0])
        red_max_scaled = np.amax(red_ref_scaled[0])
        
        # Convert to type 'uint16'
        from rasterio import uint16
        blue_scaled = blue_ref_scaled[i].astype(uint16)
        green_scaled = green_ref_scaled[i].astype(uint16)
        red_scaled = red_ref_scaled[i].astype(uint16)
        nir_scaled = nir_ref_scaled[i].astype(uint16)
        
        # New name for exported image
        img_name_ref_scaled = (Results_folder + AnalyticMS_files[i]).replace('.tif','_Ref.tif')
        
        # Write band calculations to a new unscaled GeoTIFF file with same band order (BGRN)
        with rasterio.open(img_name_ref_scaled, 'w', **kwargs) as dst:
                dst.write_band(1, blue_scaled)
                dst.write_band(2, green_scaled)
                dst.write_band(3, red_scaled)
                dst.write_band(4, nir_scaled)
         # Comparing min & max values before/after scaling, with the first image red band
        if i == 0:
            print("With scale factor: {}".format(scale))
            print("Before scaling: Red band reflectance from: {} to {}"\
                  .format(red_ref_min, red_ref_max))
            print("After scaling: Red band reflectance from: {} to {}"\
                  .format(red_min_scaled,red_max_scaled))
        if i == (len(images)-1):
            print("Success, {} scaled images(dtype={}) have been exported to your Results folder: {}"\
                  .format(n_images, kwargs['dtype'], Results_folder ))

Export of unscaled GeoTIFF, dtype=float64 with Reflectance values between 0.0 - 1.0

In [12]:
# Function to export unscaled images - (alternative option)
def unscaled_export(images):
    for i in range(len(images)):
        # Get the metadata of original GeoTIFF:
        meta = images[i].meta

        # Set the source metadata as kwargs we'll use to write the new data:
        # update the 'dtype' value from uint16 to float64:
        kwargs = meta
        kwargs.update(dtype='float64')

        # New name for exported image
        img_name_ref_unscaled = (Results_folder + AnalyticMS_files[i]).replace('.tif','_Ref_unscaled.tif')

        # Write band calculations to a new unscaled GeoTIFF file with same band order (BGRN)
        with rasterio.open(img_name_ref_unscaled, 'w', **kwargs) as dst:
                dst.write_band(1, band_blue_reflectance[i])
                dst.write_band(2, band_green_reflectance[i])
                dst.write_band(3, band_red_reflectance[i])
                dst.write_band(4, band_nir_reflectance[i])
        
        if i == (len(images)-1):
            print("Success, {} unscaled images(dtype={}) have been exported to your Results folder: {}"\
                  .format(n_images, kwargs['dtype'], Results_folder ))

##### Export Reflectance Images to your project folder
- For the scaled images, run the function *scaled_export()* with input(*img_list*)   


- For the unscaled images, run the function *unscaled_export()* with input(*img_list*)

Note that unscaled(*dtype=float64*) takes 4 times as much storage space as scaled(*dtype=uint16*)

For mosaicing and clipping to AOI, proceed further

In [13]:
# Uncomment and run for wanted image type:
# Export scaled images:
scaled_export(img_list)

# Export unscaled images:
#unscaled_export(img_list)

With scale factor: 10000
Before scaling: Red band reflectance from: 0.0 to 0.28485077640094947
After scaling: Red band reflectance from: 0.0 to 2848.5077640094946
Success, 7 scaled images(dtype=uint16) have been exported to your Results folder: planet_21/Results/


In [14]:
### Load image results into lists for the clip functions
Results_files = []
Results_path_scaled = []; Results_path_unscaled = []
img_list_scaled = []; img_list_unscaled = []

for (dirpath, dirnames, filenames) in walk(Results_folder):
    Results_files.extend(filenames)
    break

for filename in Results_files:
    if "Ref.tif" in filename:
        Results_path_scaled.append(Results_folder + filename)
        img_list_scaled.append(rasterio.open(Results_folder + filename))
    if "Ref_unscaled.tif" in filename:
        Results_path_unscaled.append(Results_folder + filename)
        img_list_unscaled.append(rasterio.open(Results_folder + filename))

Using rasterio.plot (a matplotlib interface) to preview the results of our mosaic.

#### Step 7. Creating the Mosaic

In [16]:
# Merge returns the mosaic & coordinate transformation information
from rasterio.merge import merge
(mosaic_scaled, transform) = merge(img_list_scaled)
(mosaic_unscaled, transform) = merge(img_list_unscaled)

Using rasterio.plot (a matplotlib interface) to preview the results of our mosaic.

In [29]:
# Preview results if desired
from rasterio.plot import show
#show(mosaic_scaled)

Before writing the mosaic out to a new GeoTIFF file, the metadata is taken from the first image/scene to represent the metadata of all input images.


Export of scaled unclipped mosaic GeoTIFF (multiplied with scale factor of 10 000 and stored as *dtype=uint16*)

In [18]:
# Function to export the unclipped scaled mosaic
def mosaic_scaled_export(images):
    # Get the metadata of original GeoTIFF:
        meta = images[0].meta
        
        # Update the original metadata to reflect the specifics of our new mosaic
        meta.update({"transform": transform,
                     "height":mosaic_scaled.shape[1],
                     "width":mosaic_scaled.shape[2]})
        
        # New name for exported scaled mosaic
        img_name_mosaic_scaled = (Results_folder + 'Ref_Mosaic.tif')
        
        # Write band calculations to a new scaled GeoTIFF file
        with rasterio.open(img_name_mosaic_scaled, 'w', **meta) as dst:
                dst.write(mosaic_scaled)

         # Comparing min & max values before/after scaling, with the first image red band
        if i == 0:
            print("With scale factor: {}".format(scale))
            print("Before scaling: Red band reflectance from: {} to {}"\
                  .format(red_ref_min, red_ref_max))
            print("After scaling: Red band reflectance from: {} to {}"\
                  .format(red_min_scaled,red_max_scaled))
        if i == (len(images)-1):
            print("Success, a scaled mosaic(dtype={}) composed of {} images have been "\
                  "exported to your Results folder: {}".format(meta['dtype'], n_images, Results_folder ))

In [19]:
# Function to export the unclipped scaled mosaic
def mosaic_unscaled_export(images):
    # Get the metadata of original GeoTIFF:
        meta = images[0].meta
        
        # Update the original metadata to reflect the specifics of our new mosaic
        meta.update({"transform": transform,
                     "height":mosaic_unscaled.shape[1],
                     "width":mosaic_unscaled.shape[2]})
        
        # update the 'dtype' value to float64:
        kwargs = meta
        meta.update(dtype='float64')
        
        # New name for exported mosaic unscaled
        img_name_mosaic_unscaled = (Results_folder + 'Ref_Mosaic_unscaled.tif')
        
        # Write band calculations to a new unscaled GeoTIFF file
        with rasterio.open(img_name_mosaic_unscaled, 'w', **meta) as dst:
                dst.write(mosaic_unscaled)

        if i == (len(images)-1):
            print("Success, a unscaled mosaic(dtype={}) composed of {} images have been "\
                  "exported to your Results folder: {}".format(meta['dtype'], n_images, Results_folder ))

##### Export Reflectance Mosaic to your Results folder
- For the scaled mosaic, run the function *mosaic_scaled_export()* with input(*img_list*)   


- For the unscaled images, run the function *mosaic_unscaled_export()* with input(*img_list*)

Note that unscaled(*dtype=float64*) takes 4 times as much storage space as scaled(*dtype=uint16*)

For clipped reflectance mosaic to AOI, proceed further

In [20]:
# Uncomment and run for wanted mosaic type:
# Export scaled mosaic:
mosaic_scaled_export(img_list)

# Export unscaled mosaic:
#mosaic_unscaled_export(img_list)

Success, a scaled mosaic(dtype=uint16) composed of 7 images have been exported to your Results folder: planet_21/Results/


#### Step 8. Clip the Mosaic to AOI Boundaries
A mask is used to clip the mosaic to the extents of our area of interest (AOI).

For this operation rasterio's sister-library fiona will read in our AOI (as a GeoJSON file).
Just as rasterio is used to manipulate raster data, fiona works similarly on vector data. Where rasterio represents raster imagery as numpy arrays, fiona represents vector data as GeoJSON-like Python dicts.

After the GeoJSON is read, the geometry of the AOI is extracted, with the geometry as dict key.

##### A note about Coordinate Reference Systems

Before the AOI can be applied to the mosaic, the Coordinate Reference System (CRS) need to match.
This can be checked by reading the crs attribute of the Collection object generated by *fiona.open()*.

In this example, the CRS of the AOI is: *EPSG:4326*.

While the CRS for the images is: *EPSG:32722*.


Before the clip can be applied, the geometry of the AOI needs to be transformed to match the CRS of the imagery.
Luckily, fiona is smart enough to apply the necessary mathematical transformation to a set of coordinates in order to convert them to new values: 
apply *fiona.transform.transform_geom* to the AOI geometry to do this, specifying the GeoJSON's CRS as the source CRS, and the imagery's CRS as the destination CRS.

In [21]:
# Reading AOI filename - (assumes just one file in the AOI folder)
for (dirpath, dirname, filename) in walk(aoi_folder):
    aoi_filename = filename[0]
    break
    
# Add file path
aoi_path = aoi_folder + aoi_filename
print(aoi_path) # Verify correct AOI file

planet_21/AOI/LCC_AOI.geojson


In [22]:
# Checking CRS of images and AOI
aoi_info = fiona.open(aoi_path)
mosaic_crs = img_list[0].meta['crs']
aoi_crs = aoi_info.crs['init']
    
print("Mosaic CRS:", mosaic_crs)
print('-------------------\n' "AOI CRS: {}".format(aoi_crs))

Mosaic CRS: EPSG:32722
-------------------
AOI CRS: epsg:4326


In [23]:
# Use fiona to open the original AOI GeoJSON
with fiona.open(aoi_path) as mt:
    aoi = [feature["geometry"] for feature in mt]

# Transfrom AOI CRS to match mosaic CRS
from fiona.transform import transform_geom
transformed_coords = transform_geom(str(aoi_crs), str(mosaic_crs), aoi[0])
aoi = [transformed_coords]

In [24]:
# import rasterio's mask tool
from rasterio.mask import mask

# apply mask with crop=True to cut to boundary
with rasterio.open('planet_21/Results/Ref_Mosaic.tif') as mosaic:
    clipped_scaled, transform = mask(mosaic, aoi, crop=True)
    
with rasterio.open('planet_21/Results/Ref_Mosaic_unscaled.tif') as mosaic:
    clipped_unscaled, transform = mask(mosaic, aoi, crop=True)

In [ ]:
# Watch results
#show(clipped_scaled)

#### Step 9. Export results
Lastly, the result is saved as a final output GeoTIFF, scaled and clipped to selected AOI.

In [25]:
# save the output to a final GeoTIFF
def mosaic_scale_clip_export(images):
    # use the metadata from our original mosaic
    meta = images[0].meta.copy()

    # update metadata with new, clipped mosaic's boundaries
    meta.update({"transform": transform,
        "height":clipped_scaled.shape[1],
        "width":clipped_scaled.shape[2]})

    # write the clipped scaled mosaic to a GeoTIFF
    with rasterio.open(Results_folder + "Ref_Mosaic_Clip.tif", 'w', **meta) as dst:
        dst.write(clipped_scaled)
    
    print("Success, a scaled clipped mosaic(dtype={}) composed of {} images have been "\
    "exported to your Results folder: {}".format(meta['dtype'], n_images, Results_folder ))

In [26]:

def mosaic_unscale_clip_export(images):
    # use the metadata from our original mosaic
    meta = img_list_unscaled[0].meta.copy()

    # update metadata with new, clipped mosaic's boundaries
    meta.update({"transform": transform,
        "height":clipped_scaled.shape[1],
        "width":clipped_scaled.shape[2]})

    # update the 'dtype' value to float64:
    kwargs = meta
    kwargs.update(dtype='float64')

    # write the clipped unscaled mosaic to a GeoTIFF
    with rasterio.open(Results_folder + "Ref_Mosaic_Clip_unscaled.tif", 'w', **kwargs) as dst:
        dst.write(clipped_unscaled)
    
    print("Success, a unscaled clipped mosaic(dtype={}) composed of {} images have been "\
    "exported to your Results folder: {}".format(kwargs['dtype'], n_images, Results_folder ))

##### Export clipped Reflectance Mosaics to your Results folder
- For the scaled mosaics, run the function *mosaic_scale_clip_export()* with input(*img_list*)   


- For the unscaled mosaics, run the function *mosaic_unscale_clip_export()* with input(*img_list*)   

Note that unscaled(*dtype=float64*) takes 4 times as much storage space as scaled(*dtype=uint16*)

In [27]:
# Uncomment and run for wanted mosaic type:
# Export scaled clipped mosaic:
mosaic_scale_clip_export(img_list)

# Export unscaled clipped mosaic:
#mosaic_unscale_clip_export(img_list)

Success, a scaled clipped mosaic(dtype=uint16) composed of 7 images have been exported to your Results folder: planet_21/Results/
